# Environment Setup

This notebook captures system information for benchmarking and saves it to `environment.json`.


In [ ]:
import json, platform, subprocess, psutil, numpy as np
from pathlib import Path

info = {}
info['cpu_model'] = next((l.split(':',1)[1].strip() for l in open('/proc/cpuinfo') if 'model name' in l), platform.processor())
info['cpu_count'] = psutil.cpu_count()
info['ram_gb'] = round(psutil.virtual_memory().total / 1024**3, 2)
info['os'] = platform.platform()
info['python_version'] = platform.python_version()
info['blas_libraries'] = np.__config__.get_info('blas_opt_info').get('libraries', [])
info['lapack_libraries'] = np.__config__.get_info('lapack_opt_info').get('libraries', [])
import stim, mqt.ddsim, mqt.core
info['stim_version'] = getattr(stim, '__version__', None)
info['mqt_dd_version'] = getattr(mqt.ddsim, '__version__', None)
info['mqt_core_version'] = getattr(mqt.core, '__version__', None)
try:
    import pybind11
    info['pybind11_version'] = pybind11.__version__
except Exception:
    info['pybind11_version'] = None
info['quasar_commit'] = subprocess.check_output(['git','rev-parse','HEAD']).decode().strip()
def run_cmd(cmd):
    try:
        return subprocess.check_output(cmd, text=True).splitlines()[0]
    except Exception:
        return None
info['compiler'] = run_cmd(['cc','--version']) or run_cmd(['gcc','--version'])
info['cmake_version'] = run_cmd(['cmake','--version'])
Path('environment.json').write_text(json.dumps(info, indent=2))
info


Environment information saved to `environment.json`.


## Benchmark Circuits

The table below lists benchmark circuits used in QuASAr.

In [ ]:
from benchmarks.circuits import (
    clifford_ec_circuit,
    ripple_add_circuit,
    vqe_chain_circuit,
    random_hybrid_circuit,
    recur_subroutine_circuit,
)
import inspect
from pathlib import Path
from IPython.display import Markdown


def src_link(func):
    file = Path(inspect.getsourcefile(func)).relative_to(Path.cwd())
    line = inspect.getsourcelines(func)[1]
    return f"[generator]({file}#L{line})"

circuits = [
    ("Clifford-EC", clifford_ec_circuit, "3-qubit bit-flip error correction"),
    ("Ripple-Add", ripple_add_circuit, "Ripple-carry adder for two 4-bit registers"),
    ("VQE-Chain", vqe_chain_circuit, "VQE ansatz with linear entanglement chain"),
    ("Random-Hybrid", random_hybrid_circuit, "Random mix of Clifford and T gates"),
    ("Recur-Subroutine", recur_subroutine_circuit, "Circuit with repeated subroutine layers"),
]

rows = ["| Circuit | Qubits | Gates | Description | Source |", "|---|---|---|---|---|"]
for name, fn, desc in circuits:
    circ = fn()
    rows.append(f"| {name} | {circ.num_qubits} | {circ.num_gates} | {desc} | {src_link(fn)} |")
Markdown("
".join(rows))
